In [4]:
import h2o
from h2o.automl import H2OAutoML

### Utilize agr a abordagem de meta-learning do H2O: AutoML. Esta abordagem automatiza o processo de pesquisa de um modelo de ML, testando em sequência diferentes algoritmos e configs, c intervenção humana mín. Corra o AutoML durante o tempo pretendido (e.g. 1h) e compare o mlhr modelo obtido por este processo c o mlhr modelo obtido de entre os exercícios anteriores:

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,50 mins 19 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Alice_Dias_d0aocp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.461 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [5]:
# Carregamento dos dados
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

# Remover colunas irrelevantes
train = train.drop(["C1", "id"])
test = test.drop(["C1", "id"])

# Definir target e features
target = "satisfaction"
features = [col for col in train.columns if col != target]

# Converter target para categórico, se ainda não for
train[target] = train[target].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
# Executar AutoML por tempo (ex: 600 segundos = 10 minutos)
aml = H2OAutoML(
    max_runtime_secs=600,            # ou use max_models=10 para limitar por nº de modelos
    seed=42,
    nfolds=5,
    keep_cross_validation_predictions=True
)

# Treinar
aml.train(x=features, y=target, training_frame=train)

# Ver leaderboard
lb = aml.leaderboard
print(lb)

# Ver melhor modelo
best_model = aml.leader
print(best_model.model_id)

AutoML progress: |█
10:08:36.420: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_AllModels_3_AutoML_1_20250416_100836     0.995459  0.081977   0.994599               0.0373714  0.15895   0.0252652
StackedEnsemble_AllModels_2_AutoML_1_20250416_100836     0.995449  0.0820051  0.994587               0.0373152  0.159035  0.0252921
StackedEnsemble_AllModels_1_AutoML_1_20250416_100836     0.995447  0.0820234  0.994584               0.0375719  0.159063  0.025301
StackedEnsemble_BestOfFamily_4_AutoML_1_20250416_100836  0.995352  0.0829329  0.994481               0.0380266  0.159859  0.025555
StackedEnsemble_BestOfFamily_2_AutoML_1_20250416_100836  0.995349  0.082892   0.994481               0.038037   0.159831  0.0255461
StackedEnsemble_BestOfFamily_1_AutoML_1_20250416_100

#### 1. O mlhr modelo encontrado pelo AutoML é do mm tp do mlhr modelo encontrado nos exs anteriores? Se s, quais foram as diferenças observadas a nível de config?
O mlhr modelo encontrado pelo AutoML foi tbm um Stacked Ensemble, cm no modelo treinado manualmente.
No entanto, o AutoML mlhrou ligeiramente a performance geral, combinando um conj + alargado e variado de modelos base e testando diferentes configs automaticamente.
A diferença principal está no facto de o ensemble do AutoML usar + modelos base e mlhres combinações, enqt o ensemble manual só utilizou DRF e GBM.

#### 2. Cm se comparam os modelos, em termos de performance?
Em termos de performance, o ensemble manual (Exercício 4) superou ligeiramente o mlhr modelo do AutoML, c uma AUC de 0.9993 contra 0.9955, e LogLoss significativamente + baixo (0.0457 vs 0.0819).

Isto mostra q, apesar do AutoML ser extremamente eficaz pra encontrar bons modelos automaticamente, o ensemble construído manualmente c DRF e GBM, treinado c afinação cuidadosa, conseguiu mlhr desempenho final neste problema específico.